In [1]:
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt
import scipy.sparse as sps
import scipy.optimize as sco

import import_ipynb
from Polynome2p1 import init_X
from Polynome2p1 import S
from Polynome2p1 import trapezes
from Polynome2p1 import epsi_S
from Polynome2p1 import affichage_residus

importing Jupyter notebook from Polynome2p1.ipynb
importing Jupyter notebook from Fonctions_Test.ipynb


# Polynome de degré $n = 2p + 1$ 

# Algorithme

In [2]:
def Jp(p,f,h,argf,X,epsilon):
    # calcule la jacobienne
    i = np.linspace(0,p-1,p)
    theta_i = 2*(p-i)*np.pi/(2*p+1)
    D_alpha = 2*p*np.cos(p*theta_i)/(np.cos(theta_i)-1) + np.sin(p*theta_i)/np.sin(theta_i) * 2 * (p - 1/(np.cos(theta_i)-1))
    
    i = np.linspace(1,p,p)
    eta_i = (2*(p-i)+1)*np.pi/(2*p+1)
    D_beta = 2*p*np.cos(p*eta_i)/(np.cos(eta_i)+1) + np.sin(p*eta_i)/np.sin(eta_i) * 2* (p + 1/(np.cos(eta_i)+1))

    feps = max( np.max(f(X,h,*argf)),epsilon) #cf partie 4
    return  np.sqrt(feps) * sps.diags(np.concatenate((D_alpha,D_beta))).toarray()

def b_beta (p,f,h,argf,beta,x,epsilon):
    # calcule la fonction b[beta] au point x
    res = 0
    for i in range (p+1):
        somme = (-1)**(i+p)*np.sqrt(max(f(beta[i],h,*argf),epsilon)/(1-beta[i])) 
        pro = 1
        for j in range (p+1):
            if i != j :
                pro *=  (x - beta[j])/(beta[i]-beta[j])
        somme *= pro
        res  += somme
    return res
  

def a_alpha (p,f,h,argf,alpha,x,epsilon):
    # calcule la fonction a[alpha] au point x
    res = 0
    for i in range (p+1):
        somme = (-1)**(i+p)*np.sqrt(max(f(alpha[i],h,*argf),epsilon)/(alpha[i])) 
        pro = 1
        for j in range (p+1):
            if i != j :
                pro *=  (x - alpha[j])/(alpha[i]-alpha[j])
        somme *= pro
        res += somme
    return res
     
def f_theta(X,p,f,h,argf,epsilon):
    # fonction à minimiser - [a[alpha](beta), b[beta](alpha)]
    alpha = X[:p]
    alpha_1 = np.concatenate((alpha,[1]))
    beta = X[p:]
    beta_0 = np.concatenate(([0],beta))
    return np.concatenate(( b_beta(p,f,h,argf,beta_0,alpha,epsilon), a_alpha(p,f,h,argf,alpha_1,beta,epsilon) ))

In [3]:
def Newton_Raphson (X0,p,f,h,argf,epsilon,s=None,eps=1e-12,itermax=500):
    # reconstruit le polynome le plus interpolant
    epsi = epsi_S(p)
    #initialisation
    X = np.copy(X0)
    X = np.concatenate((X[:p],X[p+2:]))
    k=0
    err=2*eps
    Residus = []
    
    while err > eps and itermax > k :
        X0 = np.copy(X)
        #print(f_theta(X,p,f,h,epsilon))
        d = npl.solve(Jp(p,f,h,argf,X,epsilon),f_theta(X,p,f,h,argf,epsilon))
        X = X - d
        if s != None or s != 0 :
            X = S(p,epsi,X)
        #print(f_theta(X,p,f,h,epsilon))
        k+=1
        err = npl.norm(X-X0)/npl.norm(X0)
        Residus += [err]

    return X,k,Residus

In [4]:
def pn(x,ap,bp): 
    #retourne le polynome à partir de la décompo de Lukas
    return x * ap**2 + (1-x) * bp**2
    
def approxh(x,p,f,h,argf,epsilon,init=None,s=None,ResidusNewton=None):
    #X = sco.fsolve(f_theta,np.array([1/4,3/4]), args=(p,f,h,epsilon))
    #X = S(p,epsi,X)
    if init is not None : 
        if isinstance(init,np.ndarray):
            X0 = init
        else : 
            X0 = init(p)
    else :
        X0 = init_X(p)
    X,itermax,residus = Newton_Raphson(X0,p,f,h,argf,epsilon,s)
    if ResidusNewton is not None and ResidusNewton != 0:
        affichage_residus(residus,p,f,h,epsilon)
    X  = np.concatenate(( X[:p],[1],[0],X[p:]))

    ap = a_alpha(p,f,h,argf,X[:p+1],x,epsilon)
    bp = b_beta(p,f,h,argf,X[p+1:],x,epsilon)
    
    return pn(x,ap,bp),[X,a_alpha,b_beta],itermax